# Trace blobs (e.g. nuclei, cells, ...) over the z-axis

Code written and conceptualized largely by Nathan De Fruyt (nathan.defruyt@kuleuven.be, nathan.defruyt@gmail.com) with initial leads by Wim Thiels (wim.thiels@kuleuven.be) in the Beets and Jelier labs. 

The algorithm is simple, but functional for minimal goals:
1. **version 3 edit:** instead of blob recognition, I threshold on the percentile of I values (i.e. also intrinsic background correction)
1. **version 4 edit:** splitting blobs that are larger than normal (95% percentile?)
1. **version 5 edit:** still to figure out splitting, but already 3D image labeling + better thresholding
1. **version 6 edit:** segment anything algorithm instead of skimage



* next, I deviated from Wim's advice and went on to work with the blob's 
    1. **center coordinates
    1. **mean intensity value
    1. **radius** (restricted to 20 pixels, as this appeared to be towards the higher end of nucleus radii -- adapt this!)

To this, the program:
1. determines **common blob labels** based on how near they are (max displacement of center = 10 pixels in x/y direction, max rise of 5 planes)
1. renders one line per blob for the plane with the **highest intensity value**

Each step (1. blob identification, 2. blob labelling, 3. summary) are rendered in separate .csv files. 
Blob identification takes the longest (a few hours for a day of pictures). The subseding steps are fast.

**Parameters can therefore be adapted** in the second and third step without consideration. 

Do think about changing parameters to the first step.

___Questions are welcome, optimization of the algorithm too.___

In [1]:
import sys

sigma = 7

## for flp-14::gfp
# channel = 1
# threshold = 99.8

# min_radius = 8
# min_z_radius = 3
# max_radius = 40

## for dmsr-3::gfp
channel = 1
threshold = 99.5

min_z_radius = 3
min_radius = 10
max_radius = 35

background_threshold = 20
split_ratio = 2/3

# threshold, background_threshold, min_radius, max_radius, split_ratio = list(map(float, sys.argv[1:]))
print(f'You indicated that nuclear localisation marker is in the channel {channel} of your image\n\n')
print(f'Initiating analysis using arguments: \n\tminimum radius = {min_radius},\n\tminimum nr. planes = {min_z_radius}\n\tmaximal radius = {max_radius},\n\tthreshold = {threshold}th percentile,\n\n\tbackground is calculated as mean of all values in the {background_threshold}th percentile\n\tall objects that have a length/width ratio > {split_ratio} will be split in two')

You indicated that nuclear localisation marker is in the channel 1 of your image


Initiating analysis using arguments: 
	minimum radius = 10,
	minimum nr. planes = 3
	maximal radius = 35,
	threshold = 99.5th percentile,

	background is calculated as mean of all values in the 20th percentile
	all objects that have a length/width ratio > 0.6666666666666666 will be split in two


In [2]:
## general math and system modules/functions
from math import sqrt, atan, tan, cos, sin
import numpy as np
import os
import glob
import itertools
import tkinter as tk
from tkinter import filedialog
from tqdm import tqdm
import shutil as sh

## parallellization!
from multiprocessing import Pool, cpu_count

## data formatting!
import pandas as pd

## image import and processing module functions
import czifile as cfile
from skimage import data, data, measure, exposure
from skimage.measure import label, regionprops_table, regionprops
from skimage.morphology import closing
from skimage.segmentation import clear_border
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.color import rgb2gray
from skimage.filters import gaussian, laplace, threshold_otsu
import cv2 as cv
import PIL
import tifffile as tf
from scipy.spatial.distance import pdist, squareform

import string

## plotting modules
import plotly
import plotly.express as px
import matplotlib.pyplot as plt
from matplotlib.pyplot import Axes

## 1. Find data

First I adapted some existing functions to more easily check and handle data either here in jupyter notebook or to the purpose of an application. 

In [3]:
# some easier to handle functions for reading image data
def readczi(filepath, channel):
    img = cfile.imread(filepath)
    print(f' >>> reading channel {channel} of {img.shape[1]}')
    return(img[0, channel-1, :img.shape[2], :img.shape[3], :img.shape[4], 0])

## 2. Render ___summary___ (mean, max, ...) intensity

In [4]:
def get_img_and_Idf_from_imstack(imstack, sigma, threshold, split_ratio = 2/3):
    
    ## take laplacian of the gaussian of the image - make sigma wide enough for good smoothing
    img_lp = laplace(gaussian(imstack, sigma = sigma))

    ## threshold on percentile - very strict, yet slightly permissive threshold of 99.8
    thresh = np.percentile(img_lp, threshold)
    # otsu = threshold_otsu(img)
    img_bw = img_lp > thresh

    ## close the edges and label adhering regions
    img_cls = closing(img_bw)
    img_lbl = label(img_cls)

    ## subtract the background from the image (everything that's below the threshold is considered background)
    bg = np.mean(img_lp < thresh)
    img_bg = imstack - bg
    img_bg[img_bg < 0] = 0

    ## measure features
    Idf = pd.DataFrame(regionprops_table(label_image=img_lbl, intensity_image=img_bg, properties = ('label', 'intensity_mean', 'centroid', 'area')))
    
    ## relabel columns
    Idf = Idf.rename(columns = {'label': "ID", 'intensity_mean': "I", 'area': "area", 'centroid-0': "z", 'centroid-1': "y", 'centroid-2': "x"})
    
    ## calculate radius
    Idf['r'] = list(map(lambda x: np.cbrt(3*x/(4*np.pi)), Idf['area']))
#     Idf['ID'] = range(0, len(Idf.index))
#     Idf.index = Idf['ID']
#     Idf['r'] = (Idf['r_x'] + Idf['r_y'])/2
    
    return(Idf, img_bg)
    

In [5]:
## function to save an image as tiff
def save_tiff(filepath, layer, dpi = 600, toplayer_alpha = .2):
    if layer == 1 or layer == 'both':
        img = readczi(filepath, channel = 1)
        mip = (np.log2(np.max(img)) - np.log2(np.max(img, axis = 0)))/np.log2(np.max(img))
    if layer == 2 or layer == 'both':
        blue = gaussian(readczi(filepath, channel = 2), sigma = 2)
        mipb = np.log2(np.max(blue)) - np.log2(np.max(blue, axis = 0))/np.log2(np.max(blue))
        print(f' >>> smoothed image')
   
    if layer == 'both':
        ## save composite:
        figname = filepath[:-len('.czi')] + '_c.tiff'
        fig, axes = plt.subplots(1, 1)

        axes.set_axis_off()
        plt.imshow(mip, cmap = 'gray')
        plt.imshow(mipb, alpha = toplayer_alpha, cmap = 'Purples')

        plt.tight_layout()
        plt.savefig(figname, dpi = dpi, format = 'tiff', bbox_inches ='tight')
        print(f' >>> Saved composite max intensity projection')

    if layer == 1 or layer == 'both':
        ## save green
        figname = filepath[:-len('.czi')] + '_g.tiff'
        fig, axes = plt.subplots(1, 1)

        axes.set_axis_off()
        plt.imshow(mip, cmap = 'gray')

        plt.tight_layout()
        plt.savefig(figname, dpi = dpi, format = 'tiff', bbox_inches ='tight')
        print(f' >>> Saved green max intensity projection')

    if layer == 2 or layer == 'both':
        ## save blue
        figname = filepath[:-len('.czi')] + '_b.tiff'
        fig, axes = plt.subplots(1, 1)

        axes.set_axis_off()
        plt.imshow(mipb, cmap = 'Purples')

        plt.tight_layout()
        plt.savefig(figname, dpi = dpi, format = 'tiff', bbox_inches ='tight')
        print(f' >>> Saved blue max intensity projection')


In [8]:
## don't touch this cell!!
folder = filedialog.askdirectory()
files = glob.glob(folder + '/*.czi')
    
for i, file in enumerate(files):
    ## print so we know that reading the file worked
    print(f'# Processing file {i+1}/{len(files)} ({os.path.basename(file)})')
    
    ## read file
    imstack = readczi(file, channel = 1)
    print(f' >>> read stack with {imstack.shape[0]} planes.')
    
    ## fetch blobs from image
    df, seq = get_img_and_Idf_from_imstack(imstack, sigma = sigma, threshold = threshold)
    print(f' >>> found {df.shape[0]} nuclei (?) in image stack')
    
    ## save as .csv file
    out_filename = folder + '/' + os.path.basename(file)[:-4] + '_objects.csv'
    df.to_csv(out_filename)
    print(f' >>> written objects to disk')
    
    ## save max intensity projection
#     ### composite
#     save_tiff(file, layer = 'both', dpi = 800, toplayer_alpha=.5)
#     print(f' >>> Saved composite max intensity projection')
#     ### green
#     save_tiff(file, layer = 1, dpi = 800)
#     print(f' >>> Saved green max intensity projection')
#     ### blue
#     save_tiff(file, layer = 2, dpi = 800)
#     print(f' >>> Saved blue max intensity projection')
    

# Processing file 1/1 (20230220_PHX6290 cntr_21%_2.czi)
 >>> reading channel 1 of 1
 >>> read stack with 99 planes.
 >>> found 352 nuclei (?) in image stack
 >>> written objects to disk


In [126]:
## don't touch this cell!!
## >> if you just want the maximum intensity projection
sigma = 7
threshold = 99.8

folder = filedialog.askdirectory()
files = glob.glob(folder + '/*.czi')
    
for i, file in enumerate(files):
    ## print so we know that reading the file worked
    print(f'# Processing file {i+1}/{len(files)} ({os.path.basename(file)})')
    
    ## save max intensity projection
    ### composite
    save_tiff(file, layer = 'both', dpi = 800)
    print(f' >>> Saved composite max intensity projection')
    ### green
    save_tiff(file, layer = 1, dpi = 800)
    print(f' >>> Saved green max intensity projection')
    ### blue
    save_tiff(file, layer = 2, dpi = 800)
    print(f' >>> Saved blue max intensity projection')
    

# Processing file 1/16 (20240820-PXXX_vX-IBE954_21_8.czi)
 >>> reading channel 1 of 2
 >>> reading channel 2 of 2
 >>> smoothed image
 >>> Saved composite max intensity projection
 >>> reading channel 1 of 2
 >>> smoothed image
 >>> Saved green max intensity projection
 >>> reading channel 2 of 2
 >>> smoothed image
 >>> Saved blue max intensity projection
# Processing file 2/16 (20240820-PXXX_vX-IBE954_7_1.czi)
 >>> reading channel 1 of 2
 >>> reading channel 2 of 2
 >>> smoothed image
 >>> Saved composite max intensity projection
 >>> reading channel 1 of 2
 >>> smoothed image
 >>> Saved green max intensity projection
 >>> reading channel 2 of 2


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x000001ED6B662AF0> (for post_execute), with arguments args (),kwargs {}:



KeyboardInterrupt

